In [2]:
import pickle 
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
df = px.data.gapminder().query("continent=='Europe'")
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
12,Albania,Europe,1952,55.230,1282697,1601.056136,ALB,8
13,Albania,Europe,1957,59.280,1476505,1942.284244,ALB,8
14,Albania,Europe,1962,64.820,1728137,2312.888958,ALB,8
15,Albania,Europe,1967,66.220,1984060,2760.196931,ALB,8
16,Albania,Europe,1972,67.690,2263554,3313.422188,ALB,8
...,...,...,...,...,...,...,...,...
1603,United Kingdom,Europe,1987,75.007,56981620,21664.787670,GBR,826
1604,United Kingdom,Europe,1992,76.420,57866349,22705.092540,GBR,826
1605,United Kingdom,Europe,1997,77.218,58808266,26074.531360,GBR,826
1606,United Kingdom,Europe,2002,78.471,59912431,29478.999190,GBR,826


In [4]:
fig = px.line_3d(df, x="gdpPercap", y="pop", z="year", color='country')
fig.show()

---

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_root = '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/epoch_vids/CKIM/PAMAP2/'

In [7]:
with open(f"{data_root}skeleton_label_predictions.pickle", 'rb') as handle:
    pred_skeleton_data = pickle.load(handle)

In [8]:
with open(f"{data_root}skeleton_original_seq.pickle", 'rb') as handle:
    origin_skeleton_data = pickle.load(handle)

In [9]:
sitting_skel = origin_skeleton_data['sitting']
res_sitting = sitting_skel.reshape((60, 12, 2))


In [56]:
def skel2df(skel):
  res_skel = skel.reshape((60, 12, 2))
  coor_data = []
  for t in range(60):
    for j in range(12):
      x0, y0 = res_skel[t, j, :]
      r = [t, j, x0, y0]
      coor_data.append(r)

  coor_df = pd.DataFrame(data=coor_data, columns=['frame', 'joint', 'x', 'y'])
  return coor_df

In [10]:
coor_data = []
for t in range(60):
  for j in range(12):
    x0, y0 = res_sitting[t, j, :]
    r = [t, j, x0, y0]
    coor_data.append(r)


In [11]:
coor_df = pd.DataFrame(data=coor_data, columns=['frame', 'joint', 'x', 'y'])
coor_df.head()

,frame,joint,x,y
0,0,0,0.464903,0.244539
1,0,1,0.398355,0.271544
2,0,2,0.486364,0.352121
3,0,3,0.372561,0.455150
4,0,4,0.500066,0.365422


In [12]:
fig = px.line_3d(coor_df, x="x", y="y", z="frame", color='joint')
fig.show()

In [67]:
pred_skeleton_data['label'][0]

{'id': 0, 'target': 'standing', 'prediction': 'standing'}

In [68]:
data_0 = pred_skeleton_data['data'][0]
res_data0 = data_0.reshape((60,12,2))

In [69]:
data0_df = skel2df(data_0)
data0_df.head()

,frame,joint,x,y
0,0,0,0.470852,0.265531
1,0,1,0.409962,0.289644
2,0,2,0.487302,0.370619
3,0,3,0.385111,0.462784
4,0,4,0.495703,0.385688


In [70]:
fig = px.line_3d(data0_df, x="x", y="y", z="frame", line_group='joint')
fig.show()

In [71]:
coor_df['st'] = 'original'
data0_df['st'] = 'generated'

comb_df = pd.concat([coor_df, data0_df], axis=0)
comb_df.head()

,frame,joint,x,y,st
0,0,0,0.464903,0.244539,original
1,0,1,0.398355,0.271544,original
2,0,2,0.486364,0.352121,original
3,0,3,0.372561,0.455150,original
4,0,4,0.500066,0.365422,original


In [72]:
comb_df.tail()

,frame,joint,x,y,st
715,59,7,0.373781,0.727727,generated
716,59,8,0.507165,0.790657,generated
717,59,9,0.479520,0.860482,generated
718,59,10,0.481123,0.962371,generated
719,59,11,0.447742,1.059406,generated


In [93]:
plt.figure(figsize=(6,6))

fig = px.line_3d(comb_df, x="x", y="y", z="frame", line_group='joint',  color='st', markers=True)
fig.update_layout(template='plotly_white')
fig.update_traces(marker={'size': 2})
fig.update_layout(
    scene=dict(
        xaxis=dict(showticklabels=False,),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False),
    )
)
fig.show()

<Figure size 600x600 with 0 Axes>

In [94]:
fig.write_html("/content/fig2.html")